In [10]:
#4-1 소벨 에지 검출하기

import cv2 as cv

img = cv.imread('soccer.jpg')
gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

grad_x=cv.Sobel(gray, cv.CV_32F, 1,0,ksize=3)
grad_y=cv.Sobel(gray, cv.CV_32F, 1,0,ksize=3)

sobel_x = cv.convertScaleAbs(grad_x)
sobel_y = cv.convertScaleAbs(grad_y)

edge_strength=cv.addWeighted(sobel_x, 0.5, sobel_y, 0.5, 0)

cv.imshow('Original', gray)
cv.imshow('sobel x', sobel_x)
cv.imshow('sobel y', sobel_y)
cv.imshow('edge strength,', edge_strength)

cv.waitKey(0)
cv.destroyAllWindows()  

In [11]:
#4-2캐니 에지

import cv2 as cv

img = cv.imread('soccer.jpg')
gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

canny1 = cv.Canny(gray,50,150)
canny2=cv.Canny(gray,100,200)

cv.imshow('Original', gray)
cv.imshow(' canny1', canny1)
cv.imshow('Canny2', canny2)

cv.waitKey(0)
cv.destroyAllWindows()  

In [1]:
#4-3 에지 맵에서 경계선 찾기

import cv2 as cv
import numpy as np

img = cv.imread('soccer.jpg')
gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
canny = cv.Canny(gray,100,200)

contour, hierachy=cv.findContours(canny,cv.RETR_LIST,cv.CHAIN_APPROX_NONE)

lcontour=[]
for i in range(len(contour)):
    if contour[i].shape[0]>100:
        lcontour.append(contour[i])
        
cv.drawContours(img, lcontour,-1,(0,255,0),3)

cv.imshow('Original with contours', img)
cv.imshow(' canny', canny)

cv.waitKey(0)
cv.destroyAllWindows()  


In [13]:
#4-4 허프변환을 이용해 동그라미 검출

import cv2 as cv
import numpy as np

img = cv.imread('soccer.jpg')
gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

soccers = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, 1, 200,
                          param1=150, param2=20,
                          minRadius=50, maxRadius=120)

if soccers is not None:
    soccers = np.uint16(np.around(soccers))
    for i in soccers[0]:
        cv.circle(img, (int(i[0]), int(i[1])), int(i[2]), (255, 0, 0), 2)

cv.imshow('soccer detection', img)
cv.waitKey(0)
cv.destroyAllWindows()

In [3]:
#4-5 SCLI 알고리즘으로 입력 영상을 슈퍼 화소 분할하기

import skimage
import numpy as np
import cv2 as cv

img=skimage.data.coffee()
cv.imshow('Coffe imgages', cv.cvtColor(img, cv.COLOR_RGB2BGR))

slic1=skimage.segmentation.slic(img, compactness=20, n_segments=600)
sp_img1=skimage.segmentation.mark_boundaries(img,slic1)
sp_img1=np.uint8(sp_img1*255.0)

slic2=skimage.segmentation.slic(img, compactness=40, n_segments=600)
sp_img2=skimage.segmentation.mark_boundaries(img,slic2)
sp_img2=np.uint8(sp_img1*255.0)

cv.imshow('super pixels (compact 20)', cv.cvtColor(sp_img1, cv.COLOR_RGB2BGR))
cv.imshow('super pixels (compact 40)', cv.cvtColor(sp_img2, cv.COLOR_RGB2BGR))

cv.waitKey(0)
cv.destroyAllWindows()


In [ ]:
#4-6 정규화 절단 알고리즘으로 영역 부할하기
import skimage
import numpy as np
import cv2 as cv
import time


coffee=skimage.data.coffee()

start = time.time()
slic=skimage.segmentation.slic(coffee, compactness=20, n_segments=600, start_label=1)
g=skimage.future.graph.rag_mean_color(coffee, slic, mode='similarity')
ncut=skimage.future.graph.cut_normalized(slic,g)
print(coffee.shape, 'Coffee 영상을 분할하는데 ', time.time()-start, '초 소요')

marking=skimage.segmentation.mark_boundaries(coffee, ncut)
ncut_coffee=np.uint8(marking*255.0)

cv.imshow('Normalized cut', cv.cvtColor(ncut_coffee, cv.COLOR_RGB2BGR))

cv.waitKey()
cv.destroyAllWindows()

In [ ]:
#4-7 GrabCut을 이용해 물체 분할하기

import cv2 as cv
import numpy as np

img = cv.imread('soccer.jpg')         # 영상 읽기
img_show = np.copy(img)               # 붓칠된 디스플레이할 목적의 영상

mask = np.zeros((img.shape[0], img.shape[1]), np.uint8)  
mask[:] = cv.GC_PR_BGD                # 모든 화소를 배경일 것 같음으로 초기화

BrushSiz = 9                          # 붓의 크기
LColor, RColor = (255, 0, 0), (0, 0, 255)  # 파란색(물체)과 빨간색(배경)

def painting(event, x, y, flags, param):
    if event == cv.EVENT_LBUTTONDOWN:
        cv.circle(img_show, (x, y), BrushSiz, LColor, -1)       # 왼쪽 버튼 클릭하면 파란색
        cv.circle(mask, (x, y), BrushSiz, cv.GC_FGD, -1)

    elif event == cv.EVENT_RBUTTONDOWN:
        cv.circle(img_show, (x, y), BrushSiz, RColor, -1)       # 오른쪽 버튼 클릭하면 빨간색
        cv.circle(mask, (x, y), BrushSiz, cv.GC_BGD, -1)

    elif event == cv.EVENT_MOUSEMOVE and flags == cv.EVENT_FLAG_LBUTTON:
        cv.circle(img_show, (x, y), BrushSiz, LColor, -1)       # 왼쪽 버튼 클릭하고 이동하면 파란색
        cv.circle(mask, (x, y), BrushSiz, cv.GC_FGD, -1)

    elif event == cv.EVENT_MOUSEMOVE and flags == cv.EVENT_FLAG_RBUTTON:
        cv.circle(img_show, (x, y), BrushSiz, RColor, -1)       # 오른쪽 버튼 클릭하고 이동하면 빨간색
        cv.circle(mask, (x, y), BrushSiz, cv.GC_BGD, -1)

cv.imshow('Painting', img_show)

cv.namedWindow('Painting')
cv.setMouseCallback('Painting', painting)

while(True):                                # 붓칠을 끝내려면 'q' 키를 누름
    if cv.waitKey(1) == ord('q'):
        break

# 여기부터 GrabCut 적용하는 코드
background = np.zeros((1, 65), np.float64)  # 배경 히스토그램 0으로 초기화
foreground = np.zeros((1, 65), np.float64)  # 물체 히스토그램 0으로 초기화

cv.grabCut(img, mask, None, background, foreground, 5, cv.GC_INIT_WITH_MASK)
mask2 = np.where((mask == cv.GC_BGD) | (mask == cv.GC_PR_BGD), 0, 1).astype('uint8')
grab = img * mask2[:, :, np.newaxis]
cv.imshow('Grab cut image', grab)

cv.waitKey()
cv.destroyAllWindows()

In [ ]:
# 4-8: 이진 영역의 특징을 추출하는 함수 사용하기

import skimage
import numpy as np
import cv2 as cv

orig = skimage.data.horse()
img = 255 - np.uint8(orig) * 255
cv.imshow('Horse', img)

contours, hierarchy = cv.findContours(img, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

img2 = cv.cvtColor(img, cv.COLOR_GRAY2BGR)  # 컬러 디스플레이용 영상
cv.drawContours(img2, contours, -1, (255,0,255), 2)
cv.imshow('Horse with contour', img2)

contour = contours[0]

m = cv.moments(contour)  # 몇 가지 특징
area = cv.contourArea(contour)
cx, cy = m['m10']/m['m00'], m['m01']/m['m00']
perimeter = cv.arcLength(contour, True)
roundness = (4.0 * np.pi * area) / (perimeter * perimeter)
print('면적=', area, '\n중심점=(', cx, ',', cy, ')', '\n둘레=', perimeter, '\n둥근 정도=', roundness)

img3 = cv.cvtColor(img, cv.COLOR_GRAY2BGR)  # 컬러 디스플레이용 영상

contour_approx = cv.approxPolyDP(contour, 8, True)  # 직선 근사
cv.drawContours(img3, [contour_approx], -1, (0,255,0), 2)

hull = cv.convexHull(contour)  # 블록 헐
hull = hull.reshape(1, hull.shape[0], hull.shape[2])
cv.drawContours(img3, hull, -1, (0,0,255), 2)

cv.imshow('Horse with line segments and convex hull', img3)

cv.waitKey()
cv.destroyAllWindows()

C:\ProgramData\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


면적= 42390.0 
중심점=( 187.72464024534088 , 144.43640402610677 ) 
둘레= 2296.7291333675385 
둥근 정도= 0.1009842680321435
